# Lab 7

In [57]:
# Install in Docker
# apt-get update
# apt-get install -y libmagickwand-dev
# gem install rmagick

In [6]:
require 'httparty'
require 'zip'
require 'torch-rb'
require 'transformers-rb'
require "onnxruntime"
require 'rmagick'




true

## Get the image dataset (interiors of houses)

- Source: https://www.kaggle.com/datasets/mikhailma/house-rooms-streets-image-dataset/data
- Cached: https://max.io/house_data_png.zip (resized to 256x256 and converted to PNG)
- License: CC-0 Public Domain

In [7]:
# Function to download and extract the zip file
def download_and_extract_zip(url, extract_to: '.')
    puts "Downloading and extracting #{url}"
    response = HTTParty.get(url)
    Zip::File.open_buffer(response.body) do |zip_file|
        zip_file.each do |entry|
            entry.extract(File.join(extract_to, entry.name))
        end
    end
end

# Download and extract the example images
# url = "https://max.io/house_data_png.zip"
# download_and_extract_zip(url)
image_dir = 'house_data_png'
image_paths = Dir[File.join(image_dir, '*')]
# puts image_paths
puts "Extracted #{image_paths.length} images"

Extracted 5249 images


In [8]:
puts image_dir

house_data_png


In [9]:
# # Load our model
# model, transform, preprocess = OpenClip.create_model_and_transforms('ViT-B-32', pretrained: 'openai')

# # Print the model architecture
# model.eval
# puts model

In [10]:
# From notebook 07c
model = OnnxRuntime::Model.new("clip_vit.onnx")




#<OnnxRuntime::Model:0x00007ffff837d610 @session=#<OnnxRuntime::InferenceSession:0x00007ffff837d4d0 @session=#<FFI::MemoryPointer address=0x000055555988f810 size=8>, @allocator=#<FFI::MemoryPointer address=0x000055555a8babf0 size=8>, @inputs=[{:name=>"IMAGE", :type=>"tensor(float)", :shape=>["image_batch_size", 3, 224, 224]}, {:name=>"TEXT", :type=>"tensor(int32)", :shape=>["text_batch_size", 77]}], @outputs=[{:name=>"LOGITS_PER_IMAGE", :type=>"tensor(float)", :shape=>["image_batch_size", "text_batch_size"]}, {:name=>"LOGITS_PER_TEXT", :type=>"tensor(float)", :shape=>["text_batch_size", "image_batch_size"]}]>>

In [11]:
model.inputs

[{:name=>"IMAGE", :type=>"tensor(float)", :shape=>["image_batch_size", 3, 224, 224]}, {:name=>"TEXT", :type=>"tensor(int32)", :shape=>["text_batch_size", 77]}]

In [12]:
model.outputs

[{:name=>"LOGITS_PER_IMAGE", :type=>"tensor(float)", :shape=>["image_batch_size", "text_batch_size"]}, {:name=>"LOGITS_PER_TEXT", :type=>"tensor(float)", :shape=>["text_batch_size", "image_batch_size"]}]

In [13]:
model.metadata


{:custom_metadata_map=>{}, :description=>"", :domain=>"", :graph_name=>"main_graph", :graph_description=>"", :producer_name=>"pytorch", :version=>9223372036854775807}

In [14]:
a = Torch.ones(5)
a.numo

Numo::SFloat#shape=[5]
[1, 1, 1, 1, 1]

In [15]:
b = Numo::NArray.cast([1, 2, 3])
Torch.from_numo(b)

tensor([1, 2, 3], dtype: :int32)

In [16]:
# Check if paths are loaded
puts "Image paths: #{image_paths.inspect}"

# Verify directory path
image_dir = 'house_data_png'
image_paths = Dir[File.join(image_dir, '*')]
puts "Found #{image_paths.length} images"



Image paths: ["house_data_png/bath_100.png", "house_data_png/bath_1001.png", "house_data_png/bath_1003.png", "house_data_png/bath_1004.png", "house_data_png/bath_1005.png", "house_data_png/bath_1006.png", "house_data_png/bath_1007.png", "house_data_png/bath_1010.png", "house_data_png/bath_1011.png", "house_data_png/bath_1012.png", "house_data_png/bath_1013.png", "house_data_png/bath_1014.png", "house_data_png/bath_1016.png", "house_data_png/bath_1018.png", "house_data_png/bath_1025.png", "house_data_png/bath_1026.png", "house_data_png/bath_1028.png", "house_data_png/bath_1029.png", "house_data_png/bath_103.png", "house_data_png/bath_1030.png", "house_data_png/bath_1031.png", "house_data_png/bath_1034.png", "house_data_png/bath_1035.png", "house_data_png/bath_1036.png", "house_data_png/bath_1038.png", "house_data_png/bath_1039.png", "house_data_png/bath_104.png", "house_data_png/bath_1041.png", "house_data_png/bath_1044.png", "house_data_png/bath_1046.png", "house_data_png/bath_105.png"

In [17]:
image_paths.count

5249

In [26]:
first_five_image_paths = image_paths.first(1)

["house_data_png/bath_100.png"]

In [27]:

#happy path
# def magick_image_to_tensor(image)
#   # 1) Resize to 224×224
#   image = image.resize_to_fill(224, 224)

#   # 2) Convert to RGB
#   image = image.quantize(256, Magick::RGBColorspace)

#   # 3) Export pixel data in "RGB" order ([0..65535])
#   pixels = image.export_pixels(0, 0, image.columns, image.rows, "RGB")

#   # 4) Convert to float32 and scale to [0,1]
#   arr = Numo::UInt16.asarray(pixels).cast_to(Numo::Float32) / 65535.0

#   # 5) Reshape from [height, width, channels] -> [channels, height, width]
#   arr = arr.reshape(image.rows, image.columns, 3)
#   arr = arr.transpose(2, 0, 1)

#   # 6) Convert to a Torch tensor and add a batch dimension => [1, 3, 224, 224]
#   Torch.from_numo(arr).unsqueeze(0)
# end

# def get_image_embeddings(model, image_paths, batch_size: 32)
#   embeddings_list = []

#   (0...image_paths.size).step(batch_size).each do |start_idx|
#     # Get the image subset for this batch
#     batch_paths = image_paths[start_idx, batch_size]

#     # Convert each image to Torch tensor => shape [1,3,224,224]
#     batch_tensors = batch_paths.map do |path|
#       img_tensor = magick_image_to_tensor(Magick::Image.read(path).first)
#       img_tensor
#     end

#     # Concatenate => shape [B,3,224,224]
#     batch_tensor = Torch.cat(batch_tensors, dim: 0)

#     # Convert Torch tensor -> Numo::NArray ([B,3,224,224]) for ONNXRuntime
#     image_data = batch_tensor.numo  # or Torch::Utils.to_numo(batch_tensor)

#     # ---------------------------------------------------------------
#     # Provide a dummy text_input so the model doesn't complain.
#     # For example, if the model expects [B,77] integer IDs for text.
#     # (Adjust shape and type as your model needs; this is just a placeholder.)
#     batch_size_actual = batch_tensor.shape[0]   # or batch_paths.size
#     dummy_text = Numo::Int32.zeros(batch_size_actual, 77)
#     # ---------------------------------------------------------------

#     # Predict with image_input + text_input
#     outputs = model.predict({
#       "image_input" => image_data,
#       "text_input"  => dummy_text
#     })

#     # If "logits_per_image" is your desired embedding output
#     batch_embeddings = outputs["logits_per_image"]
#       puts batch_embeddings.class

#     # Convert embeddings back to Torch (optional)
#     b = Numo::NArray.cast(batch_embeddings)
#     # Torch.from_numo(b)
#     batch_embeddings_torch = Torch.from_numo(b)

#     embeddings_list << batch_embeddings_torch
#   end

#   # Concatenate everything into a single [N, ...] tensor
#   Torch.cat(embeddings_list, dim: 0)
# end

In [28]:

def magick_image_to_tensor(image)
  image = image.resize_to_fill(224, 224)
  image = image.quantize(256, Magick::RGBColorspace)
  pixels = image.export_pixels(0, 0, image.columns, image.rows, "RGB")
  arr = Numo::UInt16.asarray(pixels).cast_to(Numo::Float32) / 65535.0
  arr = arr.reshape(image.rows, image.columns, 3)
  arr = arr.transpose(2, 0, 1)
  # Convert to tensor and normalize using CLIP's expected mean/std
  tensor = Torch.from_numo(arr)
  mean = Torch.tensor([0.48145466, 0.4578275, 0.40821073]).reshape(3, 1, 1)
  std = Torch.tensor([0.26862954, 0.26130258, 0.27577711]).reshape(3, 1, 1)
  ((tensor - mean) / std).unsqueeze(0)
    
      # Add debug prints
  puts "Array shape after reshape: #{arr.shape}"
  puts "First few values before normalization: #{arr[0, 0, 0..5]}"
  
  tensor = Torch.from_numo(arr)
  mean = Torch.tensor([0.48145466, 0.4578275, 0.40821073]).reshape(3, 1, 1)
  std = Torch.tensor([0.26862954, 0.26130258, 0.27577711]).reshape(3, 1, 1)
  normalized = ((tensor - mean) / std).unsqueeze(0)
  
  puts "First few values after normalization: #{normalized[0, 0, 0, 0..5]}"
  normalized
end



# def get_image_embeddings(model, image_paths, batch_size: 32)
#   embeddings_list = []
  
#   (0...image_paths.size).step(batch_size).each do |start_idx|
#     batch_paths = image_paths[start_idx, batch_size]
#     batch_tensors = batch_paths.map do |path|
#       img_tensor = magick_image_to_tensor(Magick::Image.read(path).first)
#       img_tensor
#     end
    
#     batch_tensor = Torch.cat(batch_tensors, dim: 0)
#     image_data = batch_tensor.numo
    
#     # If your ONNX has *only* "image_input":
#     outputs = model.predict({ "image_input" => image_data })

#     # Otherwise, if you had a two-input model, remove or adjust text_input:
#     # outputs = model.predict({
#     #   "image_input" => image_data,
#     #   "text_input"  => Numo::Int32.zeros(batch_tensor.shape[0], 77)
#     # })

#     # Grab the output embeddings
#     batch_embeddings = outputs["image_features"] 
#     # If nil, try alternate keys
#     if batch_embeddings.nil?
#       puts "Available keys: #{outputs.keys}"
#       batch_embeddings = outputs["image_embeddings"] || 
#                         outputs["visual_features"] ||
#                         outputs["visual_embeddings"]
#     end
    
#     batch_embeddings_arr = Numo::NArray.cast(batch_embeddings)
#     batch_embeddings_torch = Torch.from_numo(batch_embeddings_arr)
#     embeddings_list << batch_embeddings_torch
#   end
  
#   combined_embeddings = Torch.cat(embeddings_list, dim: 0)
#   puts "Combined shape: #{combined_embeddings.shape}"

#   # L2 normalize across the last dimension (dim=1 for [batch_size, 512])
#   norms = combined_embeddings.norm(2, 1, keepdim: true)
#   combined_embeddings / norms
# end


:magick_image_to_tensor

In [34]:
def get_image_embeddings(model, image_paths, batch_size: 32)
  embeddings_list = []
  
  (0...image_paths.size).step(batch_size).each do |start_idx|
    batch_paths = image_paths[start_idx, batch_size]
    batch_tensors = batch_paths.map do |path|
      img_tensor = magick_image_to_tensor(Magick::Image.read(path).first)
      puts img_tensor
      img_tensor
    end
    
    batch_tensor = Torch.cat(batch_tensors, dim: 0)
    image_data = batch_tensor.numo
    
    # Create dummy text inputs. Assuming text tokens are integers and the length is 77
    dummy_texts = Numo::Int32.zeros(batch_tensor.shape[0], 77)
    
    # Provide both IMAGE and TEXT inputs
    outputs = model.predict({
      "IMAGE" => image_data,
      "TEXT"  => dummy_texts
    })

    # Grab the output embeddings
    batch_embeddings = outputs["LOGITS_PER_IMAGE"] 
    # Adjust the key based on your model's output
    # If nil, try alternate keys
    if batch_embeddings.nil?
      puts "Available keys: #{outputs.keys}"
      batch_embeddings = outputs["image_embeddings"] || 
                        outputs["image_embeddings"] ||
                        outputs["visual_features"] ||
                        outputs["visual_embeddings"]
    end
    
    batch_embeddings_arr = Numo::NArray.cast(batch_embeddings)
    batch_embeddings_torch = Torch.from_numo(batch_embeddings_arr)
    embeddings_list << batch_embeddings_torch
  end
  
  combined_embeddings = Torch.cat(embeddings_list, dim: 0)
  puts "Combined shape: #{combined_embeddings.shape}"

  # L2 normalize across the last dimension (dim=1 for [batch_size, 512])
  norms = combined_embeddings.norm(2, 1, keepdim: true)
  combined_embeddings / norms
end


:get_image_embeddings

In [30]:
# Then call the function
# image_embeddings = get_image_embeddings(image_paths, batch_size: 32)

#Lets do the first 5 for now
image_embeddings = get_image_embeddings(model, first_five_image_paths, batch_size: 32)
image_embeddings

Array shape after reshape: [3, 224, 224]
First few values before normalization: #<Numo::SFloat:0x00007ffff92f4328>
First few values after normalization: tensor([-0.6897, -0.8876, -0.7553, -0.6617, -0.6897, -0.6897])
tensor([[[[-0.6897, -0.8876, -0.7553,  ..., -1.7825, -1.7825, -1.7825],
          [-0.7553, -0.8876, -0.6046,  ..., -1.7825, -1.7825, -1.7825],
          [-0.7553, -0.8876, -0.6897,  ..., -1.7825, -1.7825, -1.7825],
          ...,
          [-1.7825, -1.7825, -1.7825,  ..., -1.7825, -1.7825, -1.7825],
          [-1.7825, -1.7825, -1.7825,  ..., -1.7825, -1.7825, -1.7825],
          [-1.7825, -1.7825, -1.7825,  ..., -1.7825, -1.7825, -1.7825]],

         [[-0.5199, -0.6890, -0.6220,  ..., -1.7489, -1.7489, -1.7489],
          [-0.6220, -0.6890, -0.5696,  ..., -1.7489, -1.7489, -1.7489],
          [-0.6220, -0.6890, -0.5199,  ..., -1.7489, -1.7489, -1.7489],
          ...,
          [-1.7489, -1.7489, -1.7489,  ..., -1.7489, -1.7489, -1.7489],
          [-1.7489, -1.7489, -1.

tensor([[1.]], dtype: :float64)

In [31]:
# norms = Torch.norm(image_embeddings, p: 2.0, dim: 1, keepdim: true)
# image_embeddings = image_embeddings / norms

In [32]:
puts 'hello world'

hello world


In [ ]:
# puts image_embeddings
puts image_embeddings.class

Torch::Tensor


In [ ]:
require 'numo/narray'

def normalize(embeddings)
  # 1. If 'embeddings' is actually an Array of Numo arrays, handle that:
  if embeddings.is_a?(Array) && !embeddings.empty?
    # If it's exactly one Numo array, unwrap it:
    if embeddings.size == 1 && embeddings.first.is_a?(Numo::NArray)
      embeddings = embeddings.first
    else
      # If it's multiple Numo arrays, concatenate them
      # (assuming they have the same shape except for the first dimension)
      if embeddings.all? { |x| x.is_a?(Numo::NArray) }
        embeddings = Numo::NArray.concatenate(embeddings, axis: 0)
      else
        raise "Expected an array of Numo::NArray. Got: #{embeddings.inspect}"
      end
    end
  end

  # 2. Now 'embeddings' should be a single Numo::NArray.  
  unless embeddings.is_a?(Numo::NArray)
    raise "Expected a Numo::NArray, got #{embeddings.class}"
  end

  # 3. Convert Numo array -> Torch tensor
  tensor = Torch.from_numo(embeddings)

  # 4. Compute L2 norm over the last dimension => shape is kept
  norm = (tensor * tensor).sum(-1, keepdim: true).sqrt

  # 5. Divide by norm, convert back to Numo
  (tensor / norm).numo
end

# Example usage:
image_embeddings = normalize(image_embeddings)

In [ ]:
def normalize(embeddings)
  # Convert Numo array -> Torch tensor
  tensor = Torch.from_numo(embeddings)  # shape is the same as embeddings

  # L2 norm over the last dimension
  # .sum(-1, keepdim: true) => sums squares along the last dimension
  norm = (tensor * tensor).sum(-1, keepdim: true).sqrt

  # Elementwise divide, then convert back to Numo
  (tensor / norm).numo
end

# Example usage
image_embeddings = normalize(image_embeddings)

In [50]:
def normalize_embeddings(embeddings)
  # Square each element
  squared = embeddings ** 2
  
  # Sum along the last dimension (keepdims: true maintains the dimension)
  sum_squared = squared.sum(axis: -1, keepdims: true)
  
  # Calculate square root
  norm = sum_squared ** 0.5
  
  # Divide by norm to normalize
  normalized = embeddings / norm

  normalized
end

# Use the function
image_embeddings = normalize_embeddings(image_embeddings)

ArgumentError: sum() missing 1 required positional arguments: "dim"

In [51]:
# # Normalization is required!
# image_embeddings /= image_embeddings.norm(dim: -1, keepdim: true)

In [52]:
File.open('house_data_png.marshal', 'wb') do |file|
  Marshal.dump(image_embeddings, file)
end

TypeError: no _dump_data is defined for class Torch::Tensor

In [53]:
# # Save embeddings
# File.open('house_data_png.marshal', 'wb') do |file|
#     Marshal.dump(image_embeddings.cpu.numpy, file)
# end

In [54]:
puts "#{image_embeddings.length} #{image_embeddings.shape}"


10 [10, 10]


In [55]:
# puts "#{image_embeddings.length} #{image_embeddings[0].shape}"

In [56]:
# Encodes the text to the same vector space as the images
def embed_text(text)
    tokens = OpenClip::Tokenizer.tokenize([text])
    Torch.no_grad do
        text_features = model.encode_text(tokens)
        text_features /= text_features.norm(dim: -1, keepdim: true) # Normalization is required!
        text_features
    end
end

:embed_text

In [31]:
# Function to display images
def display_images(image_paths, distances)
    image_paths.each_with_index do |path, idx|
        IRuby.display(IRuby.image(path))
        puts "👆 #{distances[idx]}"
    end
end

:display_images

In [32]:
# This will search and display nearest images given a text query
nbrs = NearestNeighbors.new(n_neighbors: 10, metric: 'cosine')
nbrs.fit(image_embeddings.cpu.numpy)

def search(text)
    text_embedding = embed_text(text)
    distances, indices = nbrs.kneighbors(text_embedding.cpu.numpy)
    nearest_images = indices[0].map { |i| image_paths[i] }
    display_images(nearest_images, distances[0])
end

NameError: uninitialized constant NearestNeighbors

In [33]:
search('large kitchen island colonial')

NoMethodError: undefined method `search' for #<Object:0x00007ffffaf83a98>

In [34]:
search('white marble shower stall')

NoMethodError: undefined method `search' for #<Object:0x00007ffffaf83a98>

In [35]:
search('red ferrari')

NoMethodError: undefined method `search' for #<Object:0x00007ffffaf83a98>

In [36]:
search('nuclear reactor')

NoMethodError: undefined method `search' for #<Object:0x00007ffffaf83a98>